# ДИПЛОМНЫЙ ПРОЕКТ "Модель прогнозирования стоимости жилья для агенства недвижимости"

<span style=color:green>Задача</span> - разработать модель, которая позволила бы агенству недвижимости обойти конкурентов по скорости и качеству сделок

## 1. ПОДГРУЗКА ДАННЫХ И НЕОБХОДИМЫХ БИБЛИОТЕК

In [1]:
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim
from fake_useragent import UserAgent
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.exc import GeocoderServiceError

In [2]:
df = pd.read_csv('data/data.csv/data.csv')

## 2. ЗНАКОМСТВО С ДАННЫМИ

In [3]:
df.head(5)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

**Описание данных:**  
* <span style=background-color:#D3D3D3>'status'</span> — статус продажи;  
* <span style=background-color:#D3D3D3>'private pool'</span> и <span style=background-color:#D3D3D3>'PrivatePool'</span> — наличие собственного бассейна;  
* <span style=background-color:#D3D3D3>'propertyType'</span> — тип объекта недвижимости;  
* <span style=background-color:#D3D3D3>'street'</span> — адрес объекта;  
* <span style=background-color:#D3D3D3>'baths'</span> — количество ванных комнат;  
* <span style=background-color:#D3D3D3>'homeFacts'</span> — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);  
* <span style=background-color:#D3D3D3>'fireplace'</span> — наличие камина;  
* <span style=background-color:#D3D3D3>'city'</span> — город;  
* <span style=background-color:#D3D3D3>'schools'</span> — сведения о школах в районе;  
* <span style=background-color:#D3D3D3>'sqft'</span> — площадь в футах;  
* <span style=background-color:#D3D3D3>'zipcode'</span> — почтовый индекс;  
* <span style=background-color:#D3D3D3>'beds'</span> — количество спален;  
* <span style=background-color:#D3D3D3>'state'</span> — штат;  
* <span style=background-color:#D3D3D3>'stories'</span> — количество этажей;  
* <span style=background-color:#D3D3D3>'mls-id'</span> и <span style=background-color:#D3D3D3>'MlsId'</span> — идентификатор MLS (Multiple Listing Service, система мультилистинга);  
* <span style=background-color:#D3D3D3>'target'</span> — цена объекта недвижимости (**целевой признак, который необходимо спрогнозировать**).

**Важные сведения о данных:**  
1. Признаки <span style=background-color:#D3D3D3>'homeFacts'</span> и <span style=background-color:#D3D3D3>'schools'</span> представляют собой словари и содержат несколько типов сведений. Необходимо десериализовать содержимое этих признаков. Если в нём есть полезные данные, необходимо создать из них новые признаки.

2. Типы жилья (может пригодиться при обработке признака <span style=background-color:#D3D3D3>propertyType</span>): 
    * **apartment** — съёмная квартира (нельзя купить); 
    * **condo** — кондоминиум (можно купить); 
    * **co-op** — квартира в жилищном кооперативе;
    * **single-family (detached, tiny home)** — односемейный (отдельный, маленький) дом;
    * **townhome, townhouse** — таунхаус;

3. Основные характеристики  жилья (может пригодиться при
обработке признака <span style=background-color:#D3D3D3>propertyType</span>): 
    * **cape cod** — черепичная крыша, облицовка деревянным сайдингом, дверь в центре здания, окна по бокам, один-два этажа; 
    * **colonial home** — минимум два этажа, симметрия, лестница в центре здания, строгий внешний вид; 
    * **contemporary** — чистые, простые линии, нейтральные цвета, натуральные текстуры;
    * **cottage** — небольшая уютная веранда, небольшие жилые помещения;
    * **craftsman** — деревянные элементы ручной работы, выступающие балки, большие колонны;
    * **Greek revival** — большие белые колонны, украшения в греческом стиле, нарядный вход;
    * **farmhouse** — прямоугольная планировка, большое крыльцо, амбарная крыша;
    * **French country** — влияние прованса, облицовка камнем, состаренный вид;
    * **Mediterranean** — белая штукатурка, дерево и камень в тёплых тонах, черепичные крыши, элементы испанских и итальянских вилл;
    * **midcentury modern** — сочетание плавных природных и строгих геометрических линий, гладкость, лаконичность, большие окна;
    * **ranch** — один этаж, низкая крыша, открытые жилые помещения, задний двор;
    * **split-level** — жилые помещения разделены короткими лестничными пролётами;
    * **Tudor** — деревянные балки, каменная кладка, асимметричная крутая двускатная крыша;
    * **Victorian (Queen Anne Houses)** — два-три и более этажей, богатый декор, крутая двускатная крыша, небольшие башенки, яркий фасад;
    * **European Houses style** — кирпич или камень, высокая крутая крыша, высокие окна, часто со ставнями, традиционные декоративные детали (фронтоны, замковые камни);
    * **log home** — деревянный дом.

4. Некоторые жаргонные сокращения — для исправления дублирования (может пригодиться при обработке признака <span style=background-color:#D3D3D3>propertyType</span>):
    * **manufactured home (house)** = **mobile** = **prefab** = **modular**; 
    * **mobile** = **mo2 le** = **mo2le**; 
    * **cabin** = **ca2 n** = **ca2n**; 
    * **midcentury** = **mid century** = **mid-century**;
    * **single family** = **single-family home**.
    
5. Некоторые термины, используемые в сфере купли-продажи недвижимости:
    * **status** — статус, состояние. 
    * **estate** — объект недвижимости. 
    * **foreclosed** (**foreclosure**) — «лишён права выкупа». *Это процесс, когда недвижимость переходит к кредитору из-за неуплаты ипотеки заёмщиком. Продажа foreclosure-недвижимости отличается от традиционных продаж.*
    * **pre foreclosure** (**pre foreclosure auction**) — «до выкупа», «аукцион до выкупа».
    * **under contract showing** (**under contract show**, **under contract**, **active under contract**) — предложение о покупке уже сделано, но не принято продавцом окончательно, и недвижимость продолжают показывать другим потенциальным покупателям.
    * **under contract backups**, **active backup**, **backup contract** — предложение о покупке уже сделано и принято продавцом, однако он открыт для резервных предложений. 
    * **contingency** — дополнительные обстоятельства (критерии, которым должен соответствовать продавец/покупатель до завершения сделки).
    * **contingency contract** — контракт на случай возникновения дополнительных обстоятельств.
    * **active contingency** — сделка уже практически состоялась, однако продавцу/покупателю для её завершения необходимо выполнить какие-то условия.
    * **insp inspection contingency** — ситуация, когда покупатель может официально проинспектировать объект недвижимости в определённый период времени.
    * **pending escape clause** — договор, предполагающий свободный выход сторон из него.
    * **pending backup wanted** — договор уже заключен, однако продавец хочет продолжать показывать недвижимость другим потенциальным покупателям и принимать резервные предложения.
    * **pending take backups** — условие, предполагающее, что продавец может принимать резерные предложения, если сделка с текущим покупателем не состоится.
    * **pending continue show** — договор уже заключен, однако продавец хочет продолжать показывать недвижимость другим потенциальным покупателям на случай, если текущая сделка не состоится.
    * **pending inspection** — «в ожидании инспекции».
    * **due diligence period** — период, в течение которого покупатель имеет право проинспектировать недвижимость и изучить всю важную для заключения сделки информацию.
    * **activated** — «активен», открыт для покупки.
    * **active with contract** — продавец уже принял предложение о покупке, однако сделка ещё не закрыта.
    * **active with offer** — продавец принял предложение о покупке, однако ищет резервные предложения на случай, если сделка с текущим покупателем не состоится.
    * **active auction**, **auction active**, **auction** — продажа недвижимости с помощью аукциона.
    * **cooling-off period** — период, в течение которого покупатель может расторгнуть договор после завершения сделки.

## 3. ОЧИСТКА ДАННЫХ

Посмотрим на количество дублирующих наблюдения в данных:

In [5]:
dubl = df.duplicated().sum()
print(f'В данных содержится {dubl} дубликатов')

В данных содержится 50 дубликатов


Удалим дубликаты:

In [6]:
df = df.drop_duplicates(ignore_index=True)

Посмотрим на количество пропусков в признаках:

In [7]:
# общщее колличество пропусков по каждому признаку
missings = df.isnull().sum()
missings

status           39917
private pool    372954
propertyType     34733
street               2
baths           106308
homeFacts            0
fireplace       274023
city                34
schools              0
sqft             40550
zipcode              0
beds             91254
state                0
stories         150673
mls-id          352193
PrivatePool     336825
MlsId            66869
target            2480
dtype: int64

In [8]:
# пропуски в признаках в процентном соотношении
for col_name, missings_count in missings.items():
    if missings_count > 0:
        percents = (missings_count / len(df)) * 100
        print(col_name, round(percents,2),'%')
        

status 10.58 %
private pool 98.89 %
propertyType 9.21 %
street 0.0 %
baths 28.19 %
fireplace 72.66 %
city 0.01 %
sqft 10.75 %
beds 24.2 %
stories 39.95 %
mls-id 93.39 %
PrivatePool 89.31 %
MlsId 17.73 %
target 0.66 %


### Признаки private pool и PrivatePool

Признаки **private pool** и **PrivatePool** содержат более 80% пропусков.

Согласно брифу Признаки **private pool** и **PrivatePool** содержат одинаковую информацию: наличие или отсутствие собственного бассейна. Вероятно, один содержит данные отсутсвующие в другом, а второй данные отсутсвующие в первом. Посмотрим на их содержание:

In [9]:
print('Уникальные значения в параметре private pool:', df['private pool'].unique())
print('Уникальные значения в параметре PrivatePool:', df['PrivatePool'].unique())

Уникальные значения в параметре private pool: [nan 'Yes']
Уникальные значения в параметре PrivatePool: [nan 'yes' 'Yes']


Признак **PrivatePool** содержит три варианта значения, в то время как **private pool** только два. Приведем к единообразию и объединим признаки:

In [10]:
df['PrivatePool'] = df['PrivatePool'].replace('yes', 'Yes')
#объединим два параметра используя метод DataFrame.combine_first, чтобы заполнить пропуски одного значением другого
df['PrivatePool'] = df['private pool'].combine_first(df['PrivatePool'])
# удалим столбец private pool
df = df.drop('private pool', axis=1)
#посмотрим на кол-во пропусков PrivatePool после объединения
print(df['PrivatePool'].isnull().sum())

332644


Пропуски в признаке **PrivatePool** указывают на отсутсвие собственного басейна, булево закодируем признаки, пусть наличие басейна будет *True*, а отсутсвие - *False*:

In [11]:
#заменим NaN значение на False
df['PrivatePool'] = df['PrivatePool'].fillna(False)
#заменим значение Yes на True
df['PrivatePool'] = df['PrivatePool'].replace('Yes', True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377135 entries, 0 to 377134
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337218 non-null  object
 1   propertyType  342402 non-null  object
 2   street        377133 non-null  object
 3   baths         270827 non-null  object
 4   homeFacts     377135 non-null  object
 5   fireplace     103112 non-null  object
 6   city          377101 non-null  object
 7   schools       377135 non-null  object
 8   sqft          336585 non-null  object
 9   zipcode       377135 non-null  object
 10  beds          285881 non-null  object
 11  state         377135 non-null  object
 12  stories       226462 non-null  object
 13  mls-id        24942 non-null   object
 14  PrivatePool   377135 non-null  bool  
 15  MlsId         310266 non-null  object
 16  target        374655 non-null  object
dtypes: bool(1), object(16)
memory usage: 46.4+ MB


Посмотрим на дубликаты после преобразования признака **PrivatePool**:

In [13]:
df.duplicated().sum()

0

### Признак fireplace

In [14]:
#посмотрим на общее кол-во уникальных значений признака
df['fireplace'].str.lower().nunique()

1652

In [15]:
#посмотрим на распределение уникальных значений признака
df['fireplace'].str.lower().value_counts().head(30)

yes                          71209
1                            14544
2                             2432
not applicable                1993
fireplace                      847
3                              564
living room                    433
location                       399
wood burning                   311
gas/gas logs                   300
no                             289
fireplace yn                   287
special features               279
1 fireplace                    274
0                              271
familyrm                       246
fireplace features             239
great room                     207
wood                           206
gas logs, great room           188
ceiling fan                    186
living room, wood burning      185
4                              184
family room, wood burning      149
familyrm, gas logs             146
gas                            134
gas logs, in great room        112
gas logs                       107
gas logs, in living 

Признак **fireplace** содержит множество различных вариаций, однако приобладающим значением является '*yes*'. По сути, его значимость заключается в определении наличия или отсутсвия камина. Давайте его бинарно закодируем. Наличие камина будет True, отсутсвие - False:

In [16]:
df['fireplace'] = df['fireplace'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no'] and not pd.isna(x) else False)

In [17]:
# посмотрим на соотношение вариаций признака после кодирования
df['fireplace'].value_counts()


False    274295
True     102840
Name: fireplace, dtype: int64

### Признак propertyType

Посмотрим на количество уникальных значений в **propertyType**:

In [18]:
df['propertyType'].nunique()

1280

Посмотрим на распределение уникальных значений:

In [19]:
df['propertyType'].value_counts().head(40)

single-family home               92199
Single Family                    62867
Single Family Home               31728
condo                            25964
lot/land                         20526
Condo                            16561
townhouse                        11456
Land                             10934
multi-family                      7900
Condo/Townhome/Row Home/Co-Op     7701
Townhouse                         6936
Traditional                       5913
coop                              3265
Multi Family                      2793
High Rise                         1823
Ranch                             1781
mobile/manufactured               1618
Detached, One Story               1614
Single Detached, Traditional      1581
Contemporary                      1557
Multi-Family Home                 1501
1 Story                           1234
Colonial                          1205
Mobile / Manufactured             1066
Contemporary/Modern               1000
2 Stories                

Видим, что многие значения различаются по форме написания, но суть есть одно. Преобразуем их, а также сформируем значение *Other* из значений не описывающих тип недвижимости:

In [20]:
#преобразуем все значения к строчному написанию
df.propertyType = df.propertyType.str.lower()
#приведем написание к единой форме,  так же заменим yes и unknown на Other
df.propertyType = df.propertyType \
.str.replace('single-family home','single family') \
.str.replace('single family home','single family') \
.str.replace('yes','Other') \
.str.replace('unknown','Other')
    
#выведем количество уникальных значений после преобразования
print('Количество уникальных значений после преобразования:{}'.format(df['propertyType'].nunique()))


Количество уникальных значений после преобразования:1269


В признаке много значений выполненных в форме перечисления через запятую, так же большое количество синонимов. Приобразуем эти данные:

In [21]:
#избавимся от избыточной информации
df['propertyType'] = df['propertyType'].str.split(',').str[0]
#функция преобразования синонимов
def syn_rename(value):
    for key, syn_val in synonyms.items():
        if value in syn_val:
            return key
    return 'other'

#словарь синонимов
synonyms = {
    'single_family_home': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached', 'singlefamilyresidence', 'single wide', 'single-wide mobile with land',
        'two story', 'one story', 'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'],
    'multi_family_home': [
        'multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex', 'multi_level', 'multi family', '2 unit condo', '2-story'],
    'condo': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise'],
    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'],
    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'Flats', 'studio'],
    'land': [
        'lot/land', 'land'],
    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home', 'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story', 'mobile manu - double wide'],
    'miscellaneous': [
        'miscellaneous'],
    'ranch': [
        'ranch', 'one story'],
    'modern': [
        'contemporary', 'contemporary/modern', "modern", 'mid-century modern', 'modern farmhouse', 'modernist'],
    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older', 'historic vintage', 'historic'],
    'other': [
        'Other', 'english', 'urban contemporary', 'other style', 'florida', 'farms/ranches', 'carriage house', 'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor', 'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house', 'farm house', 'hi ranch', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', 'victorian', '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam', 'manuf/mobile', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home', '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement', 'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor', 'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch','residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio', 'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii', 'coastal modern', 'coastal two story', 'mountain contemporary', 'key west', 'high ranch', 'end unit']
}

#преобразуем синонимы в нашем признаке
df['propertyType']= df['propertyType'].apply(syn_rename)

#смотрим количество пропусков и уникальных значений после приобразования
print(f'количество пропусков:{df.propertyType.isna().sum()}\n'
      f'количество уникальных значений:{df.propertyType.nunique()}\n'
      f'уникальные значения:\n{df.propertyType.value_counts()}')

количество пропусков:0
количество уникальных значений:12
уникальные значения:
single_family_home    195937
condo                  56958
other                  51584
land                   31460
townhouse              18543
multi_family_home      12349
mobile_home             3551
modern                  3252
ranch                   2033
apartment               1308
miscellaneous             96
historical                64
Name: propertyType, dtype: int64


### Признак stories

Признак **stories** содержит информацию об количестве этажей. Посмотрим на количество уникальных значений, пропусков:

In [22]:
df['stories'].nunique()

348

### Признак street!!!!!

Признак **street** содержит информацию об адресе объекта. Посмотрим нуждается ли он в оптимизации:

In [23]:
df['street'].value_counts()

Address Not Disclosed        672
Undisclosed Address          517
(undisclosed Address)        391
Address Not Available        175
Unknown Address               72
                            ... 
1737 Parkview Green Cir        1
14890 Rockridge Ln             1
497 Kingswood Dr               1
202 Seacrest Beach Blvd W      1
5983 Midcrown Dr               1
Name: street, Length: 337076, dtype: int64

In [24]:
df['street'].isna().sum()

2

Мы имеем два пропуска и множестно значений обозначающих, что адрес не разглашается, либо недоступен, либо неизвестен. Примим идею, что все эти значения есть суть одно, объединим их, отнесем к ним пропуски:

In [25]:
df1 = df.copy()

In [26]:
df1['street'].value_counts(ascending=False).head(10)

Address Not Disclosed    672
Undisclosed Address      517
(undisclosed Address)    391
Address Not Available    175
Unknown Address           72
2103 E State Hwy 21       57
11305 Gulf Fwy            54
17030 Youngblood Rd.      38
NE 58th Cir               34
9470 Lancaster Rd. SW     32
Name: street, dtype: int64

In [27]:
#список синонимов
disclosed_adress = {
    'Undisclosed Address':['Address Not Disclosed', 'Undisclosed Address','(undisclosed Address)','Address Not Available', 'Unknown Address']
}
#функция замены синонимов
def undisclosed_Address(value):
    for key, val in disclosed_adress.items():
        if value in val:
            return key
    return value
#заменяем синонимы
df1['street'] = df1['street'].apply(undisclosed_Address)    
#заменяем пропуски на Undisclosed Address        
df1['street'] = df1['street'].fillna('Undisclosed Address')        

df1['street'].value_counts()

Undisclosed Address          1829
2103 E State Hwy 21            57
11305 Gulf Fwy                 54
17030 Youngblood Rd.           38
NE 58th Cir                    34
                             ... 
1737 Parkview Green Cir         1
14890 Rockridge Ln              1
497 Kingswood Dr                1
202 Seacrest Beach Blvd W       1
5983 Midcrown Dr                1
Name: street, Length: 337072, dtype: int64

In [28]:
df1['street'].nunique()

337072

In [35]:
df1['street'] = df1['street'].str.lower()

In [36]:
df1['street'].nunique()

336262

### Признак city!!!

Есть пропуски попробуем востановить по признаку **zipcode** при помощи **geopy**:

In [37]:
df_zip = df1.copy()

In [40]:
user_agent = UserAgent().random
geolocator = Nominatim(user_agent=user_agent, timeout=10)

def get_coordinates(address, retry_count=3):
    if retry_count <= 0:
       return None

    try:
        location = geolocator.geocode(address)
        if location:
            return (location.address)
        else:
            return None
    except (GeocoderTimedOut, GeocoderUnavailable, GeocoderServiceError):
        return get_coordinates(address, retry_count - 1)

df_zip['location'] = df_zip['zipcode'].apply(get_coordinates)

In [41]:
df_zip.head(5)

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,location
0,Active,single_family_home,240 heather ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",True,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,False,611019,"$418,000","Southern Pines, Moore County, North Carolina, ..."
1,for sale,single_family_home,12911 e heroy ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",False,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,False,201916904,"$310,000","Degučiai, Žemaičių Naumiesčio seniūnija, Šilut..."
2,for sale,single_family_home,2005 westridge rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",True,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,True,FR19221027,"$2,895,000","Terrasini, Palermo, Sicilia, 90049, Italia"
3,for sale,single_family_home,4311 livingston ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",True,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,False,14191809,"$2,395,000","Kulli küla, Raasiku vald, Harju maakond, 75205..."
4,for sale,land,1524 kiscoe st,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,False,861745,"$5,000","Palm Bay, Brevard County, Florida, 32908, Unit..."


In [43]:
df_zip[df_zip['city'].isna()]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,location
7830,Active,single_family_home,13155 w highway 316,2,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",False,NaN,"[{'rating': ['1', '4', '3', '3', '6', '6'], 'd...",1016,32686,1,FL,2,NaN,False,A10445273,"$1,495,000","Marion County, Florida, 32686, United States"
21529,Active,land,se 117th ter,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,32668,NaN,FL,NaN,NaN,False,F10183087,"$49,900",None
26466,Active,land,blk 4 n america lareedo rd lot 8,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,78045,NaN,TX,NaN,NaN,False,20183582,"$417,718",None
60990,Active,land,ocala,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34474,NaN,FL,NaN,NaN,False,F10011873,"$31,334",None
81509,Active,land,unditermined unditermined,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34432,NaN,FL,NaN,NaN,False,A10652556,"$28,000",None
122337,Active,land,sw149 ter,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34481,NaN,FL,NaN,NaN,False,A10697411,"$18,000",None
123481,Active,land,26484 deer rd,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': ['6', '5', '7', '7', '2', '10', 'N...",0,33955,NaN,FL,NaN,NaN,False,A10326590,"$4,000",None
130554,For sale,single_family_home,20003 mar rojo y mar egeo,Bathrooms: 6,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",False,NaN,"[{'rating': ['5/10', 'NA', '4/10'], 'data': {'...","Total interior livable area: 5,905 sqft",20003,5 bd,MA,NaN,A10761803,True,NaN,"$1,650,000",None
131266,Active,land,19212 roosevelt,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': ['5', '4', '6', 'NR', '5', 'NR', '...",0,33954,NaN,FL,NaN,NaN,False,A10741077,"$4,200",None
151954,Active,land,ocala,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34474,NaN,FL,NaN,NaN,False,F10011860,"$31,334",None


In [45]:
df_zip['city'].isna().sum()

34

Видим, что попытка решить через **geopy** не увенчалась успехом. Пропусков всего 34, заполним их в ручном режиме проверяя zip-code через https://tools.usps.com/  сверяя результат с кодом штата признака **state**:

In [46]:
df_zip_1 = df_zip.copy()

In [60]:
df_zip_1[df_zip_1['city'].isna()]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,location
26466,Active,land,blk 4 n america lareedo rd lot 8,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,78045,NaN,TX,NaN,NaN,False,20183582,"$417,718",None
60990,Active,land,ocala,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34474,NaN,FL,NaN,NaN,False,F10011873,"$31,334",None
81509,Active,land,unditermined unditermined,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34432,NaN,FL,NaN,NaN,False,A10652556,"$28,000",None
122337,Active,land,sw149 ter,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34481,NaN,FL,NaN,NaN,False,A10697411,"$18,000",None
123481,Active,land,26484 deer rd,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': ['6', '5', '7', '7', '2', '10', 'N...",0,33955,NaN,FL,NaN,NaN,False,A10326590,"$4,000",None
130554,For sale,single_family_home,20003 mar rojo y mar egeo,Bathrooms: 6,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",False,NaN,"[{'rating': ['5/10', 'NA', '4/10'], 'data': {'...","Total interior livable area: 5,905 sqft",20003,5 bd,MA,NaN,A10761803,True,NaN,"$1,650,000",None
131266,Active,land,19212 roosevelt,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': ['5', '4', '6', 'NR', '5', 'NR', '...",0,33954,NaN,FL,NaN,NaN,False,A10741077,"$4,200",None
151954,Active,land,ocala,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,34474,NaN,FL,NaN,NaN,False,F10011860,"$31,334",None
170335,Active,single_family_home,2678 calistoga ave,7.5,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",False,NaN,"[{'rating': ['2', '4', '6', '4', '3', '10', 'N...",0,34741,7,FL,2,NaN,False,A10416812,"$675,000",None
171290,Active,land,block 4 n america laredo rd lot 9,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,NaN,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,78045,NaN,TX,NaN,NaN,False,20190240,"$582,750",None


In [55]:
df_zip_1.at[7830,'city'] = 'Reddick'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[26466,'city'] = 'LAREDO'
df_zip_1.at[60990,'city'] = 'OCALA'
df_zip_1.at[81509,'city'] = 'DUNNELLON'
df_zip_1.at[122337,'city'] = 'OCALA'
df_zip_1.at[123481,'city'] = 'PUNTA GORDA'
df_zip_1.at[131266,'city'] = 'PORT CHARLOTTE'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'
df_zip_1.at[21529,'city'] = 'MORRISTON'

In [57]:
df_zip_1.loc[[7830, 21529]]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,location
7830,Active,single_family_home,13155 w highway 316,2,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",False,Reddick,"[{'rating': ['1', '4', '3', '3', '6', '6'], 'd...",1016,32686,1,FL,2,NaN,False,A10445273,"$1,495,000","Marion County, Florida, 32686, United States"
21529,Active,land,se 117th ter,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",False,MORRISTON,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,32668,NaN,FL,NaN,NaN,False,F10183087,"$49,900",None


### Признак Status

Признак **Status** содержит информацию о статусе продажи объекта. Посмотрим на количество уникальных значений, пропусков:

In [30]:
print(f'количество пропусков:{df.status.isna().sum()}\n'
      f'количество уникальных значений:{df.status.nunique()}\n'
      f'распределение значений признака:\n{df.status.value_counts()}')

количество пропусков:39917
количество уникальных значений:159
распределение значений признака:
for sale                156058
Active                  105207
For sale                 43464
foreclosure               6425
New construction          5474
                         ...  
Contingent   No Show         1
Coming soon: Oct 24.         1
Coming soon: Oct 21.         1
Coming soon: Nov 14.         1
Coming soon: Dec 23.         1
Name: status, Length: 159, dtype: int64


In [31]:
print(f"уникальные значения:{df['status'].sort_values().unique()}\n")

уникальные значения:[' / auction' 'A Active' 'Accepted Offer' 'Accepting backups' 'Active'
 'Active - Auction' 'Active - Contingent' 'Active Backup'
 'Active Contingency' 'Active Contingent' 'Active Option'
 'Active Option Contract' 'Active Under Contract'
 'Active With Contingencies' 'Active With Offer' 'Active with Contract'
 'Active/Contingent' 'Apartment for rent' 'Auction' 'Auction - Active'
 'Back On Market' 'Back on Market' 'Backup' 'Backup Contract' 'C'
 'C Continue Show' 'CT Insp - Inspection Contingency' 'Closed'
 'Coming soon: Dec 1.' 'Coming soon: Dec 10.' 'Coming soon: Dec 11.'
 'Coming soon: Dec 12.' 'Coming soon: Dec 13.' 'Coming soon: Dec 14.'
 'Coming soon: Dec 15.' 'Coming soon: Dec 16.' 'Coming soon: Dec 18.'
 'Coming soon: Dec 2.' 'Coming soon: Dec 20.' 'Coming soon: Dec 23.'
 'Coming soon: Dec 24.' 'Coming soon: Dec 25.' 'Coming soon: Dec 27.'
 'Coming soon: Dec 3.' 'Coming soon: Dec 4.' 'Coming soon: Dec 5.'
 'Coming soon: Dec 6.' 'Coming soon: Dec 7.' 'Coming soo

Видим, что некоторые значения дублируются (различаются сокращением, регистром), так же присутствует признак *Coming soon*. Пропуски заменим на *No info*. Выполним необходимые преобразования:

In [32]:
#нормализуем регистра
df['status'] = df['status'].str.lower()
#словарь синонимов
stasus_synonims = {
    'pending':['option pending' ,  'pending (do not show)', 'pending ab',  'pending fe', 'pending offer approval',  'pending sh', 'pending w/escape clause', 'p', 'pending'],
    
    'pending Continue To Show':['pending   continue to show', 'pending - continue to show', 'pending continue to show', 'pending, continue to show', 'pending   continue to show   financing'],
      
    'Pending With Contingencies':['pending with contingencies','pending w/ cont.'],
    
    'Pending Inspection':['pending inspection', 'pending w/insp finance', 'pending in', 'pi'],
    
    'Pending Sale':['pending sale', 'p pending sale'],
    
    'Pending Taking Backups':['pending taking backups', 'pending - paking backups', 'pending bring backup', 'pending - backup offer requested','pending backups wanted', 'pending take backups', 'pending w/backup wanted'],
    
    'auction':[' / auction', 'a active', 'active', 'active - auction', 'auction - active', 'auction'],
    
    'closed':['c', 'closed'],  
    
    'Coming soon: Dec':['coming soon: dec 1.', 'coming soon: dec 10.', 'coming soon: dec 11.', 'coming soon: dec 12.', 'coming soon: dec 13.', 'coming soon: dec 14.',
                        'coming soon: dec 15.', 'coming soon: dec 16.', 'coming soon: dec 18.', 'coming soon: dec 2.', 'coming soon: dec 20.', 'coming soon: dec 23.',
                        'coming soon: dec 24.', 'coming soon: dec 25.', 'coming soon: dec 27.', 'coming soon: dec 3.', 'coming soon: dec 4.', 'coming soon: dec 5.',
                        'Coming soon: Dec 6.', 'Coming soon: Dec 7.', 'Coming soon: Dec 9.'],
    
    'Coming soon: Nov':['coming soon: nov 11.', 'coming soon: nov 12.', 'coming soon: nov 13.', 'coming soon: nov 14.', 'coming soon: nov 17.', 'coming soon: nov 19.',
                        'coming soon: nov 21.', 'coming soon: nov 22.', 'coming soon: nov 23.', 'coming soon: nov 25.', 'coming soon: nov 26.', 'coming soon: nov 27.',
                        'coming soon: nov 28.', 'coming soon: nov 29.', 'coming soon: nov 30.', 'coming soon: nov 5.', 'coming soon: nov 8.'],
    
    'Coming soon: Oct':['coming soon: oct 21.', 'coming soon: oct 24.', 'coming soon: oct 29.', 'coming soon: oct 30.'], 
    
    'Under Contract':['under contract', 'under contract'],
    
    'Under Contract Backups':['under contract backups', 'under contract taking back up offers', 'under contract w/ bckp'],
    
    'Under Contract Showing':['under contract   showing', 'uc continue to show', 'under contract - show',  'under contract show'],
    
    'Pre-foreclosure':[ 'pre-foreclosure', 'pf']
    
    }
#функция замены синонимов
def status_rename(value):
  for key, val in stasus_synonims.items():
    if value in val:
      return key
  return value
#заменяем синонимы
df['status'] = df['status'].apply(status_rename)
#заменяем пропуски на No info
df['status'] = df['status'].fillna('No info')

Смотрим на результаты после преобразования:

In [33]:
print(f'количество пропусков:{df.status.isna().sum()}\n'
      f'количество уникальных значений:{df.status.nunique()}\n'
      f'распределение значений признака:\n{df.status.value_counts()}')

количество пропусков:0
количество уникальных значений:83
распределение значений признака:
for sale                 199524
auction                  107160
No info                   39917
foreclosure                6768
pending                    6423
                          ...  
conting accpt backups         1
contingent   no show          1
conditional contract          1
contract                      1
backup                        1
Name: status, Length: 83, dtype: int64


### MlsId и mls-id

Параметры **MlsId** и **mls-id** -  идентификаторы система мультилистинга, содержат уникальный идентификационный номер объекта. Параметр не влияет на стоимость объекта, его следует удалить: